<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_02_3_llm_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 2: Code Generation**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 2 Material

* Part 2.1: Prompting for Code Generation [[Video]](https://www.youtube.com/watch?v=HVId6kYKKgQ) [[Notebook]](t81_559_class_02_1_dev.ipynb)
* Part 2.2: Handling Revision Prompts [[Video]](https://www.youtube.com/watch?v=APpV46tplXA) [[Notebook]](t81_559_class_02_2_multi_prompt.ipynb)
* **Part 2.3: Using a LLM to Help Debug** [[Video]](https://www.youtube.com/watch?v=VPqSNb38QK0) [[Notebook]](t81_559_class_02_3_llm_debug.ipynb)
* Part 2.4: Tracking Prompts in Software Development [[Video]](https://www.youtube.com/watch?v=oUFUuYfvXZU) [[Notebook]](t81_559_class_02_4_software_eng.ipynb)
* Part 2.5: Limits of LLM Code Generation [[Video]](https://www.youtube.com/watch?v=dKtRI0LZSyY) [[Notebook]](t81_559_class_02_5_code_gen_limits.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: not using Google CoLab


# 2.3: Using a LLM to Help Debug

LLMs can help you debug both the code you create and the code you generate to fulfill your requests. In this part, you will see how to use an LLM as an assistant to help debug a Python program.

## Conversational Code Generation

We will continue to use the conversational code generation function provided in Module 2.2.



In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'
TEMPLATE = """The following is a friendly conversation between a human and an
AI to generate Python code. If you have notes about the code, place them before
the code. Any nots about execution should follow the code. If you do mix any
notes with the code, make them comments. Add proper comments to the code.
Sort imports and follow PEP-8 formatting.

Current conversation:
{history}
Human: {input}
Code Assistant:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def start_conversation():
    # Initialize the OpenAI LLM with your API key
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory()
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def generate_code(conversation, prompt):
    print("Model response:")
    output = conversation.invoke(prompt)
    display_markdown(output['response'], raw=True)


## A Buggy Pi Approximator

To see an example of how you can make use of LLM-enabled debugging, consider the following code to use the [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) method to estimate [Pi](https://en.wikipedia.org/wiki/Pi). We need to fix several issues with this code. We can request the LLM to help us debug. This code, when executed, produces the following error:

```
NameError: name 'xrange' is not defined
```

In [3]:
import random

def monte_carlo_pi(num_samples):
    inside_circle = 0

    for _ in xrange(num_samples):
        x, y = random.random(), random.random()  # Generate random point (x, y)
        if x*2 + y*2 <= 1:
            inside_circle += 1  # Check if the point is inside the quarter circle

    pi_approximation = 4 * inside_circle / num_samples  # Calculate approximation of Pi
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

NameError: name 'xrange' is not defined

When we ask the LLM to help us debug this code, we should provide as much detail as possible. I usually like to produce a prompt in the following format:

```
I am trying to debug the following code:

... provide code here...

However, I am getting the following error:

... add the error here, provide stack trace ...

```

In [4]:
conversation = start_conversation()
generate_code(conversation, """
I am trying to debug the following code:

import random

def monte_carlo_pi(num_samples):
    inside_circle = 0

    for _ in xrange(num_samples):
        x, y = random.random(), random.random()  # Generate random point (x, y)
        if x*2 + y*2 <= 1:
            inside_circle += 1  # Check if the point is inside the quarter circle

    pi_approximation = 4 * inside_circle / num_samples  # Calculate approximation of Pi
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

However, I am getting the following error:

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-10-c7b4356f1718> in <cell line: 16>()
     14 # Example usage
     15 num_samples = 1000000  # Number of random points to generate
---> 16 approximated_pi = monte_carlo_pi(num_samples)
     17 print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

<ipython-input-10-c7b4356f1718> in monte_carlo_pi(num_samples)
      4     inside_circle = 0
      5
----> 6     for _ in xrange(num_samples):
      7         x, y = random.random(), random.random()  # Generate random point (x, y)
      8         if x*2 + y*2 <= 1:

NameError: name 'xrange' is not defined

""")

C:\Users\yrobi\AppData\Local\Temp\ipykernel_27832\2023780929.py:30: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(


Model response:


It looks like you're encountering a `NameError` because `xrange` is not defined in Python 3. In Python 3, you should use `range` instead of `xrange`. Additionally, there are a couple of other improvements we can make to the code, such as correcting the condition for checking if the point is inside the quarter circle and ensuring proper formatting according to PEP-8.

Here’s the corrected code:

```python
import random

def monte_carlo_pi(num_samples):
    """
    Estimate the value of Pi using the Monte Carlo method.

    Parameters:
    num_samples (int): The number of random samples to generate.

    Returns:
    float: The estimated value of Pi.
    """
    inside_circle = 0  # Counter for points inside the quarter circle

    for _ in range(num_samples):  # Use range instead of xrange
        x, y = random.random(), random.random()  # Generate random point (x, y)
        # Check if the point is inside the quarter circle
        if x**2 + y**2 <= 1:  
            inside_circle += 1  # Increment counter if inside

    # Calculate approximation of Pi
    pi_approximation = 4 * inside_circle / num_samples  
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")
```

### Notes on Execution:
- Make sure you are running this code in a Python 3 environment since `xrange` is not available in Python 3.
- The condition for checking if the point is inside the quarter circle has been corrected to `if x**2 + y**2 <= 1:` to properly reflect the equation of a circle.
- The code is now formatted according to PEP-8 standards, including proper spacing and comments.

In this case, the LLM decided to be an overachiever because I only asked it about the specific error I was getting. However, the LLM provided me with two issues, one of which was the error I encountered. The LLM identified these two issues:

* It looks like you're using Python 3, where xrange has been replaced by range.

* Also, there's a mistake in the condition to check if the point is inside the quarter circle. It should be ```x ** 2 + y ** 2 <= 1``` instead ```of x*2 + y*2 <= 1```.

The LLM also provided a corrected code for me to copy/paste.

## Testing the Corrected Code

Now, we can test the corrected code and see that it works properly.

In [5]:
import random

def monte_carlo_pi(num_samples):
    """
    Estimate the value of Pi using the Monte Carlo method.

    Parameters:
    num_samples (int): The number of random samples to generate.

    Returns:
    float: The estimated value of Pi.
    """
    inside_circle = 0  # Counter for points inside the quarter circle

    for _ in range(num_samples):  # Use range instead of xrange
        x, y = random.random(), random.random()  # Generate random point (x, y)
        # Check if the point is inside the quarter circle
        if x**2 + y**2 <= 1:  
            inside_circle += 1  # Increment counter if inside

    # Calculate approximation of Pi
    pi_approximation = 4 * inside_circle / num_samples  
    return pi_approximation

# Example usage
num_samples = 1000000  # Number of random points to generate
approximated_pi = monte_carlo_pi(num_samples)
print(f"Approximated Pi with {num_samples} samples: {approximated_pi}")

Approximated Pi with 1000000 samples: 3.138624


## LLMs Explaining Code

LLMs are also very adept at explaining code. As you work through this course, you will see that the assignments use a submission function I named "submit." This submission function uses HTTP and API calling techniques that are not covered by this course. However, if you are interested in what the "submit" function does, you can ask the LLM.

In [6]:
# Start a new conversation
conversation = start_conversation()
generate_code(conversation, """
Could you please explain what the following code does?

import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
""")

Model response:


The provided code defines a function `submit` that allows a user to submit an assignment consisting of data (either images or pandas DataFrames) to a specified API endpoint. The function checks for the validity of the input parameters, encodes the source file and data, and sends a POST request to the server. Below is the cleaned-up version of the code with proper comments and PEP-8 formatting.

```python
import base64
import os
import requests
import pandas as pd
from PIL import Image
from io import BytesIO

# This function submits an assignment. You can submit an assignment as many times as you like; only the final
# submission counts. The parameters are as follows:
# data - List of pandas DataFrames or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file. This must have "_class1" as part of its name.
#               The number must match your assignment number. For example, "_class2" for class assignment #2.
def submit(data, key, no, source_file=None):
    # Check if source_file is provided; if not, use the current file if available
    if source_file is None and '__file__' not in globals():
        raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None:
        source_file = __file__

    # Validate that the source_file contains the correct class suffix
    suffix = '_class{}'.format(no)
    if suffix not in source_file:
        raise Exception('{} must be part of the filename.'.format(suffix))

    # Read and encode the source file
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')

    # Validate the file extension
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception("Source file is {} must be .py or .ipynb".format(ext))

    # Prepare the payload for submission
    payload = []
    for item in data:
        if isinstance(item, Image.Image):
            # If the item is an image, convert it to PNG and encode it
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            # If the item is a DataFrame, convert it to CSV and encode it
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})

    # Send the POST request to the API
    response = requests.post(
        "https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key': key},
        json={'payload': payload, 'assignment': no, 'ext': ext, 'py': encoded_python}
    )

    # Check the response status and print the result
    if response.status_code == 200:
        print("Success: {}".format(response.text))
    else:
        print("Failure: {}".format(response.text))
```

### Notes about execution:
- Ensure that the `requests`, `pandas`, and `PIL` libraries are installed in your Python environment.
- The function requires a valid API key and a correctly formatted source file name to work properly.
- The `data` parameter should contain either images (as `PIL.Image.Image` objects) or pandas DataFrames.
- The API endpoint used in the code is specific to Heaton Research; ensure that it is active and accessible.

As you can see, the LLM explained my "submit" function.

## Improving Code with LLMs

You can also request that a LLM improve your code. You can mention specific improvements you seek, such as removing unused or redundant imports, sorting the imports, and adhering to PEP-8 for your code formatting. In the following code, I request that the LLM improve my submit function.

In [8]:
conversation = start_conversation()
generate_code(conversation, """
Could you please suggest and implement any improvements to the following code?

import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))
""")

Model response:


Here are some improvements and notes on the provided code:

1. **Import Optimization**: Group imports from the same library together.
2. **Error Handling**: Add more specific error messages and handle potential exceptions that could occur during the request.
3. **Code Readability**: Improve readability by breaking down complex lines into simpler statements and adding more comments.
4. **Function Documentation**: Add a docstring to the function to explain its purpose, parameters, and expected behavior.
5. **Constants**: Use uppercase for constants like the URL.
6. **Resource Management**: Use `with` statement for handling `BytesIO` to ensure proper resource management.

Here's the revised version of the code:

```python
import base64
import io
import os

import numpy as np
import pandas as pd
import requests
from PIL import Image

# Constants
SUBMISSION_URL = "https://api.heatonresearch.com/assignment-submit"

def submit(data, key, no, source_file=None):
    """
    Submit an assignment with given data, student key, and assignment number.
    
    Parameters:
        data (list): List of pandas DataFrames or PIL Images to be submitted.
        key (str): Student key for authentication.
        no (int): Assignment number, should be a valid class number.
        source_file (str, optional): Path to the source Python or IPYNB file.
            If not provided, it attempts to use the current file.
    
    Raises:
        Exception: If the source file is not specified in a Jupyter notebook environment.
        Exception: If the source file name does not contain the correct class suffix.
        Exception: If the source file extension is not .py or .ipynb.
    """
    if source_file is None and '__file__' not in globals():
        raise Exception('Must specify a filename when in a Jupyter notebook.')
    if source_file is None:
        source_file = __file__
    
    suffix = '_class{}'.format(no)
    if suffix not in source_file:
        raise Exception('{} must be part of the filename.'.format(suffix))
    
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception("Source file extension {} must be .py or .ipynb".format(ext))
    
    payload = []
    for item in data:
        if isinstance(item, Image.Image):
            with io.BytesIO() as buffered:
                item.save(buffered, format="PNG")
                payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    
    response = requests.post(SUBMISSION_URL, headers={'x-api-key': key},
                             json={'payload': payload, 'assignment': no, 'ext': ext, 'py': encoded_python})
    
    if response.status_code == 200:
        print("Success: {}".format(response.text))
    else:
        print("Failure: {}".format(response.text))
```

### Execution Notes:
- Ensure that the `requests` library is installed in your environment (`pip install requests`).
- The function now uses `isinstance()` for type checking, which is generally preferred over direct type comparison.
- The `BytesIO` object is managed using a `with` statement to ensure it is properly closed after its use.

As you can see, the LLM suggested several improvements that I will consider for future versions of this function.